<a href="https://colab.research.google.com/github/RCortez25/TinyTorch/blob/main/0.%20Foundation/01_tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#| default_exp core.tensor
#| export

import numpy as np

# Constants for memory calculations
BYTES_PER_FLOAT32 = 4  # Standard float32 size in bytes
KB_TO_BYTES = 1024  # Kilobytes to bytes conversion
MB_TO_BYTES = 1024 * 1024  # Megabytes to bytes conversion

# Tensor class

In [2]:
#| export
class Tensor:
    """Tensor - the foundation of machine learning computation.

    This class provides the core data structure for all ML operations:
    - data: The actual numerical values (NumPy array)
    - shape: Dimensions of the tensor
    - size: Total number of elements
    - dtype: Data type (float32)

    All arithmetic, matrix, and shape operations are built on this foundation.
    """
    def __init__(self, data):
        """Create a new tensor from data."""
        self.data = np.array(data, dtype=np.float32)
        self.shape = self.data.shape
        self.size = self.data.size
        self.dtype = self.data.dtype

    def __repr__(self):
        "String representation for debugging"
        return f"Tensor(data={self.data}, shape={self.shape})"

    def __str__(self):
        "String representation"
        return f"Tensor({self.data})"

    def numpy(self):
        "Return the NumPy array"
        return self.data

    def memory_footprint(self):
        """Calculate exact memory usage in bytes.

        Systems Concept: Understanding memory footprint is fundamental to ML systems.
        Before running any operation, engineers should know how much memory it requires.

        Returns:
            int: Memory usage in bytes (e.g., 1000x1000 float32 = 4MB)
        """
        return self.data.nbytes

    def __add__(self, other):
        """Add two tensors element-wise with broadcasting support."""
        if isinstance(other, Tensor):
            return Tensor(self.data + other.data)
        return self.data + other

    def __sub__(self, other):
        """Subtract two tensors element-wise."""

        if isinstance(other, Tensor):
            return Tensor(self.data - other.data)
        return self.data - other

    def __mul__(self, other):
        """Multiply two tensors element-wise (NOT matrix multiplication)."""
        if isinstance(other, Tensor):
            return Tensor(self.data * other.data)
        return self.data * other

    def __truediv__(self, other):
        """Divide two tensors element-wise."""
        if isinstance(other, Tensor):
            return Tensor(self.data / other.data)
        return Tensor(self.data / other)

    def matmul(self, other):
        """Matrix multiplication of two tensors."""
        if not isinstance(other, Tensor):
            raise TypeError("Both elements must be tensors")
        if other.data.ndim == 0:
            return Tensor(self * other)
        if other.data.ndim == 1:
            return Tensor(np.matmul(self.data, other.data))
        if not self.shape[-1] == other.shape[-2]:
            raise ValueError(f"Inner dimensions must match. {self.shape[-1]} ≠ {other.shape[-2]}")
        if self.data.shape == (2,2):
            lst = []
            for i in range(2):
                for j in range(2):
                    result = np.dot(self.data[i,:],other.data[:,j])
                    lst.append(result)
            return Tensor(np.array(lst).reshape(2,2))
        return Tensor(np.matmul(self.data, other.data))

    def __matmul__(self, other):
        """Enable @ operator for matrix multiplication."""
        return self.matmul(other)

    def __getitem__(self, key):
        """Enable indexing and slicing operations on Tensors."""
        return Tensor(np.array(self.data[key]))

    def reshape(self, *shape):
        """Reshape tensor to new dimensions.
        - For -1: unknown_dim = self.size // known_size in the tuple
        """
        if isinstance(shape[0], (tuple, list)):
            shape = shape[0]
        if shape[-1] == -1:
            unknown_dim = self.size // shape[0]
            shape = (shape[0],unknown_dim)
        if self.data.size != np.prod(shape):
            raise ValueError(f"Total elements must match. {self.data.size} ≠ {np.prod(shape)}")
        return Tensor(self.data.reshape(shape))

    def transpose(self, dim0=None, dim1=None):
        """Transpose tensor dimensions."""
        axes = list(range(len(self.shape)))
        if len(axes) == 1:
            return self
        if dim0 is None and dim1 is None:
            axes[-2], axes[-1] = axes[-1], axes[-2]
            return Tensor(np.transpose(self.data, axes))
        axes[dim0], axes[dim1] = axes[dim1], axes[dim0]
        return Tensor(np.transpose(self.data, axes))

    def sum(self, axis=None, keepdims=False):
        """Sum tensor along specified axis."""
        return Tensor(np.sum(self.data, axis=axis, keepdims=keepdims))

    def mean(self, axis=None, keepdims=False):
        """Compute mean of tensor along specified axis."""
        result = np.mean(self.data, axis=axis, keepdims=keepdims)
        return Tensor(result)

    def max(self, axis=None, keepdims=False):
        """Find maximum values along specified axis."""
        result = np.max(self.data, axis=axis, keepdims=keepdims)
        return Tensor(result)

In [3]:
#################### TESTS
data = Tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

# Reshape to 3D tensor (simulating batch processing)
tensor_3d = data.reshape(2, 2, 3)  # (batch=2, height=2, width=3)
assert tensor_3d.shape == (2, 2, 3)
print(tensor_3d.shape)
# Transpose for different operations
transposed = tensor_3d.transpose()  # Should transpose last two dims
transposed.shape
assert transposed.shape == (2, 3, 2)
#################### TESTS

(2, 2, 3)


# Unit tests

In [4]:
def test_unit_reduction_operations():
    """🧪 Test reduction operations."""
    print("🧪 Unit Test: Reduction Operations...")

    matrix = Tensor([[1, 2, 3], [4, 5, 6]])  # Shape: (2, 3)

    # Test sum all elements (common for loss computation)
    total = matrix.sum()
    assert total.data == 21.0  # 1+2+3+4+5+6
    assert total.shape == ()   # Scalar result

    # Test sum along axis 0 (columns) - batch dimension reduction
    col_sum = matrix.sum(axis=0)
    expected_col = np.array([5, 7, 9], dtype=np.float32)  # [1+4, 2+5, 3+6]
    assert np.array_equal(col_sum.data, expected_col)
    assert col_sum.shape == (3,)

    # Test sum along axis 1 (rows) - feature dimension reduction
    row_sum = matrix.sum(axis=1)
    expected_row = np.array([6, 15], dtype=np.float32)  # [1+2+3, 4+5+6]
    assert np.array_equal(row_sum.data, expected_row)
    assert row_sum.shape == (2,)

    # Test mean (average loss computation)
    avg = matrix.mean()
    assert np.isclose(avg.data, 3.5)  # 21/6
    assert avg.shape == ()

    # Test mean along axis (batch normalization pattern)
    col_mean = matrix.mean(axis=0)
    expected_mean = np.array([2.5, 3.5, 4.5], dtype=np.float32)  # [5/2, 7/2, 9/2]
    assert np.allclose(col_mean.data, expected_mean)

    # Test max (finding best predictions)
    maximum = matrix.max()
    assert maximum.data == 6.0
    assert maximum.shape == ()

    # Test max along axis (argmax-like operation)
    row_max = matrix.max(axis=1)
    expected_max = np.array([3, 6], dtype=np.float32)  # [max(1,2,3), max(4,5,6)]
    assert np.array_equal(row_max.data, expected_max)

    # Test keepdims (important for broadcasting)
    sum_keepdims = matrix.sum(axis=1, keepdims=True)
    assert sum_keepdims.shape == (2, 1)  # Maintains 2D shape
    expected_keepdims = np.array([[6], [15]], dtype=np.float32)
    assert np.array_equal(sum_keepdims.data, expected_keepdims)

    # Test 3D reduction (simulating global average pooling)
    tensor_3d = Tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])  # (2, 2, 2)
    spatial_mean = tensor_3d.mean(axis=(1, 2))  # Average across spatial dimensions
    assert spatial_mean.shape == (2,)  # One value per batch item

    print("✅ Reduction operations work correctly!")

if __name__ == "__main__":
    test_unit_reduction_operations()

🧪 Unit Test: Reduction Operations...
✅ Reduction operations work correctly!


In [5]:
def test_unit_shape_manipulation():
    """🧪 Test reshape and transpose operations."""
    print("🧪 Unit Test: Shape Manipulation...")

    # Test basic reshape (flatten → matrix)
    tensor = Tensor([1, 2, 3, 4, 5, 6])  # Shape: (6,)
    reshaped = tensor.reshape(2, 3)      # Shape: (2, 3)
    assert reshaped.shape == (2, 3)
    expected = np.array([[1, 2, 3], [4, 5, 6]], dtype=np.float32)
    assert np.array_equal(reshaped.data, expected)

    # Test reshape with tuple (alternative calling style)
    reshaped2 = tensor.reshape((3, 2))   # Shape: (3, 2)
    assert reshaped2.shape == (3, 2)
    expected2 = np.array([[1, 2], [3, 4], [5, 6]], dtype=np.float32)
    assert np.array_equal(reshaped2.data, expected2)

    # Test reshape with -1 (automatic dimension inference)
    auto_reshaped = tensor.reshape(2, -1)  # Should infer -1 as 3
    assert auto_reshaped.shape == (2, 3)

    # Test reshape validation - should raise error for incompatible sizes
    try:
        tensor.reshape(2, 2)  # 6 elements can't fit in 2×2=4
        assert False, "Should have raised ValueError"
    except ValueError as e:
        assert "Total elements must match" in str(e)
        assert "6 ≠ 4" in str(e)

    # Test matrix transpose (most common case)
    matrix = Tensor([[1, 2, 3], [4, 5, 6]])  # (2, 3)
    transposed = matrix.transpose()          # (3, 2)
    assert transposed.shape == (3, 2)
    expected = np.array([[1, 4], [2, 5], [3, 6]], dtype=np.float32)
    assert np.array_equal(transposed.data, expected)

    # Test 1D transpose (should be identity)
    vector = Tensor([1, 2, 3])
    vector_t = vector.transpose()
    assert np.array_equal(vector.data, vector_t.data)

    # Test specific dimension transpose
    tensor_3d = Tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])  # (2, 2, 2)
    swapped = tensor_3d.transpose(0, 2)  # Swap first and last dimensions
    assert swapped.shape == (2, 2, 2)  # Same shape but data rearranged

    # Test neural network reshape pattern (flatten for MLP)
    batch_images = Tensor(np.random.rand(2, 3, 4))  # (batch=2, height=3, width=4)
    flattened = batch_images.reshape(2, -1)  # (batch=2, features=12)
    assert flattened.shape == (2, 12)

    print("✅ Shape manipulation works correctly!")

if __name__ == "__main__":
    test_unit_shape_manipulation()

🧪 Unit Test: Shape Manipulation...
✅ Shape manipulation works correctly!


In [6]:
def test_unit_matrix_multiplication():
    """🧪 Test matrix multiplication operations."""
    print("🧪 Unit Test: Matrix Multiplication...")

    # Test 2×2 matrix multiplication (basic case)
    a = Tensor([[1, 2], [3, 4]])  # 2×2
    b = Tensor([[5, 6], [7, 8]])  # 2×2
    result = a.matmul(b)
    # Expected: [[1×5+2×7, 1×6+2×8], [3×5+4×7, 3×6+4×8]] = [[19, 22], [43, 50]]
    expected = np.array([[19, 22], [43, 50]], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test rectangular matrices (common in neural networks)
    c = Tensor([[1, 2, 3], [4, 5, 6]])  # 2×3 (like batch_size=2, features=3)
    d = Tensor([[7, 8], [9, 10], [11, 12]])  # 3×2 (like features=3, outputs=2)
    result = c.matmul(d)
    # Expected: [[1×7+2×9+3×11, 1×8+2×10+3×12], [4×7+5×9+6×11, 4×8+5×10+6×12]]
    expected = np.array([[58, 64], [139, 154]], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test matrix-vector multiplication (common in forward pass)
    matrix = Tensor([[1, 2, 3], [4, 5, 6]])  # 2×3
    vector = Tensor([1, 2, 3])  # 3×1 (conceptually)
    result = matrix.matmul(vector)
    # Expected: [1×1+2×2+3×3, 4×1+5×2+6×3] = [14, 32]
    expected = np.array([14, 32], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test shape validation - should raise clear error
    try:
        incompatible_a = Tensor([[1, 2]])     # 1×2
        incompatible_b = Tensor([[1], [2], [3]])  # 3×1
        incompatible_a.matmul(incompatible_b)  # 1×2 @ 3×1 should fail (2 ≠ 3)
        assert False, "Should have raised ValueError for incompatible shapes"
    except ValueError as e:
        assert "Inner dimensions must match" in str(e)
        assert "2 ≠ 3" in str(e)  # Should show specific dimensions

    print("✅ Matrix multiplication works correctly!")

if __name__ == "__main__":
    test_unit_matrix_multiplication()

🧪 Unit Test: Matrix Multiplication...
✅ Matrix multiplication works correctly!


In [7]:
def test_unit_arithmetic_operations():
    """🧪 Test arithmetic operations with broadcasting."""
    print("🧪 Unit Test: Arithmetic Operations...")

    # Test tensor + tensor
    a = Tensor([1, 2, 3])
    b = Tensor([4, 5, 6])
    result = a + b
    assert np.array_equal(result.data, np.array([5, 7, 9], dtype=np.float32))

    # Test tensor + scalar (very common in ML)
    result = a + 10
    assert np.array_equal(result.data, np.array([11, 12, 13], dtype=np.float32))

    # Test broadcasting with different shapes (matrix + vector)
    matrix = Tensor([[1, 2], [3, 4]])
    vector = Tensor([10, 20])
    result = matrix + vector
    expected = np.array([[11, 22], [13, 24]], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Test subtraction (data centering)
    result = b - a
    assert np.array_equal(result.data, np.array([3, 3, 3], dtype=np.float32))

    # Test multiplication (scaling)
    result = a * 2
    assert np.array_equal(result.data, np.array([2, 4, 6], dtype=np.float32))

    # Test division (normalization)
    result = b / 2
    assert np.array_equal(result.data, np.array([2.0, 2.5, 3.0], dtype=np.float32))

    # Test chaining operations (common in ML pipelines)
    normalized = (a - 2) / 2  # Center and scale
    expected = np.array([-0.5, 0.0, 0.5], dtype=np.float32)
    assert np.allclose(normalized.data, expected)

    print("✅ Arithmetic operations work correctly!")

if __name__ == "__main__":
    test_unit_arithmetic_operations()

🧪 Unit Test: Arithmetic Operations...
✅ Arithmetic operations work correctly!


In [8]:
def test_unit_tensor_creation():
    """🧪 Test Tensor creation with various data types."""
    print("🧪 Unit Test: Tensor Creation...")

    # Test scalar creation
    scalar = Tensor(5.0)
    assert scalar.data == 5.0
    assert scalar.shape == ()
    assert scalar.size == 1
    assert scalar.dtype == np.float32

    # Test vector creation
    vector = Tensor([1, 2, 3])
    assert np.array_equal(vector.data, np.array([1, 2, 3], dtype=np.float32))
    assert vector.shape == (3,)
    assert vector.size == 3

    # Test matrix creation
    matrix = Tensor([[1, 2], [3, 4]])
    assert np.array_equal(matrix.data, np.array([[1, 2], [3, 4]], dtype=np.float32))
    assert matrix.shape == (2, 2)
    assert matrix.size == 4

    # Test 3D tensor creation
    tensor_3d = Tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
    assert tensor_3d.shape == (2, 2, 2)
    assert tensor_3d.size == 8

    print("✅ Tensor creation works correctly!")

if __name__ == "__main__":
    test_unit_tensor_creation()

🧪 Unit Test: Tensor Creation...
✅ Tensor creation works correctly!


# Memory analysis

In [9]:
def analyze_memory_layout():
    """📊 Demonstrate cache effects with row vs column access patterns."""
    print("📊 Analyzing Memory Access Patterns...")
    print("=" * 60)

    # Create a moderately-sized matrix (large enough to show cache effects)
    size = 2000
    matrix = Tensor(np.random.rand(size, size))

    import time

    print(f"\nTesting with {size}×{size} matrix ({matrix.size * BYTES_PER_FLOAT32 / MB_TO_BYTES:.1f} MB)")
    print("-" * 60)

    # Test 1: Row-wise access (cache-friendly)
    # Memory layout: [row0][row1][row2]... stored contiguously
    print("\n🔬 Test 1: Row-wise Access (Cache-Friendly)")
    start = time.time()
    row_sums = []
    for i in range(size):
        row_sum = matrix.data[i, :].sum()  # Access entire row sequentially
        row_sums.append(row_sum)
    row_time = time.time() - start
    print(f"   Time: {row_time*1000:.1f}ms")
    print(f"   Access pattern: Sequential (follows memory layout)")

    # Test 2: Column-wise access (cache-unfriendly)
    # Must jump between rows, poor spatial locality
    print("\n🔬 Test 2: Column-wise Access (Cache-Unfriendly)")
    start = time.time()
    col_sums = []
    for j in range(size):
        col_sum = matrix.data[:, j].sum()  # Access entire column with large strides
        col_sums.append(col_sum)
    col_time = time.time() - start
    print(f"   Time: {col_time*1000:.1f}ms")
    print(f"   Access pattern: Strided (jumps {size * BYTES_PER_FLOAT32} bytes per element)")

    # Calculate slowdown
    slowdown = col_time / row_time
    print("\n" + "=" * 60)
    print(f"📊 PERFORMANCE IMPACT:")
    print(f"   Slowdown factor: {slowdown:.2f}× ({col_time/row_time:.1f}× slower)")
    print(f"   Cache misses cause {(slowdown-1)*100:.0f}% performance loss")

    # Educational insights
    print("\n💡 KEY INSIGHTS:")
    print(f"   1. Memory layout matters: Row-major (C-style) storage is sequential")
    print(f"   2. Cache lines are ~64 bytes: Row access loads nearby elements \"for free\"")
    print(f"   3. Column access misses cache: Must reload from DRAM every time")
    print(f"   4. This is O(n) algorithm but {slowdown:.1f}× different wall-clock time!")

    print("\n🚀 REAL-WORLD IMPLICATIONS:")
    print(f"   • CNNs use NCHW format (channels sequential) for cache efficiency")
    print(f"   • Matrix multiplication optimized with blocking (tile into cache-sized chunks)")
    print(f"   • Transpose is expensive ({slowdown:.1f}×) because it changes memory layout")
    print(f"   • This is why GPU frameworks obsess over memory coalescing")

    print("\n" + "=" * 60)

# Run the systems analysis
if __name__ == "__main__":
    analyze_memory_layout()

📊 Analyzing Memory Access Patterns...

Testing with 2000×2000 matrix (15.3 MB)
------------------------------------------------------------

🔬 Test 1: Row-wise Access (Cache-Friendly)
   Time: 7.7ms
   Access pattern: Sequential (follows memory layout)

🔬 Test 2: Column-wise Access (Cache-Unfriendly)
   Time: 12.5ms
   Access pattern: Strided (jumps 8000 bytes per element)

📊 PERFORMANCE IMPACT:
   Slowdown factor: 1.63× (1.6× slower)
   Cache misses cause 63% performance loss

💡 KEY INSIGHTS:
   1. Memory layout matters: Row-major (C-style) storage is sequential
   2. Cache lines are ~64 bytes: Row access loads nearby elements "for free"
   3. Column access misses cache: Must reload from DRAM every time
   4. This is O(n) algorithm but 1.6× different wall-clock time!

🚀 REAL-WORLD IMPLICATIONS:
   • CNNs use NCHW format (channels sequential) for cache efficiency
   • Matrix multiplication optimized with blocking (tile into cache-sized chunks)
   • Transpose is expensive (1.6×) because

# Module integration test

In [12]:
def test_module():
    """🧪 Module Test: Complete Integration

    Comprehensive test of entire module functionality.

    This final test runs before module summary to ensure:
    - All unit tests pass
    - Functions work together correctly
    - Module is ready for integration with TinyTorch
    """
    print("🧪 RUNNING MODULE INTEGRATION TEST")
    print("=" * 50)

    # Run all unit tests
    print("Running unit tests...")
    test_unit_tensor_creation()
    test_unit_arithmetic_operations()
    test_unit_matrix_multiplication()
    test_unit_shape_manipulation()
    test_unit_reduction_operations()

    print("\nRunning integration scenarios...")

    # Test realistic neural network computation
    print("🧪 Integration Test: Two-Layer Neural Network...")

    # Create input data (2 samples, 3 features)
    x = Tensor([[1, 2, 3], [4, 5, 6]])

    # First layer: 3 inputs → 4 hidden units
    W1 = Tensor([[0.1, 0.2, 0.3, 0.4],
                 [0.5, 0.6, 0.7, 0.8],
                 [0.9, 1.0, 1.1, 1.2]])
    b1 = Tensor([0.1, 0.2, 0.3, 0.4])

    # Forward pass: hidden = xW1 + b1
    hidden = x.matmul(W1) + b1
    assert hidden.shape == (2, 4), f"Expected (2, 4), got {hidden.shape}"

    # Second layer: 4 hidden → 2 outputs
    W2 = Tensor([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6], [0.7, 0.8]])
    b2 = Tensor([0.1, 0.2])

    # Output layer: output = hiddenW2 + b2
    output = hidden.matmul(W2) + b2
    assert output.shape == (2, 2), f"Expected (2, 2), got {output.shape}"

    # Verify data flows correctly (no NaN, reasonable values)
    assert not np.isnan(output.data).any(), "Output contains NaN values"
    assert np.isfinite(output.data).all(), "Output contains infinite values"

    print("✅ Two-layer neural network computation works!")

    # Test complex shape manipulations
    print("🧪 Integration Test: Complex Shape Operations...")
    data = Tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

    # Reshape to 3D tensor (simulating batch processing)
    tensor_3d = data.reshape(2, 2, 3)  # (batch=2, height=2, width=3)
    assert tensor_3d.shape == (2, 2, 3)

    # Global average pooling simulation
    pooled = tensor_3d.mean(axis=(1, 2))  # Average across spatial dimensions
    assert pooled.shape == (2,), f"Expected (2,), got {pooled.shape}"

    # Flatten for MLP
    flattened = tensor_3d.reshape(2, -1)  # (batch, features)
    assert flattened.shape == (2, 6)

    # Transpose for different operations
    transposed = tensor_3d.transpose()  # Should transpose last two dims
    assert transposed.shape == (2, 3, 2)

    print("✅ Complex shape operations work!")

    # Test broadcasting edge cases
    print("🧪 Integration Test: Broadcasting Edge Cases...")

    # Scalar broadcasting
    scalar = Tensor(5.0)
    vector = Tensor([1, 2, 3])
    result = scalar + vector  # Should broadcast scalar to vector shape
    expected = np.array([6, 7, 8], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    # Matrix + vector broadcasting
    matrix = Tensor([[1, 2], [3, 4]])
    vec = Tensor([10, 20])
    result = matrix + vec
    expected = np.array([[11, 22], [13, 24]], dtype=np.float32)
    assert np.array_equal(result.data, expected)

    print("✅ Broadcasting edge cases work!")

    print("\n" + "=" * 50)
    print("🎉 ALL TESTS PASSED! Module ready for export.")
    print("Run: tito module complete 01_tensor")

# Run comprehensive module test
if __name__ == "__main__":
    test_module()

🧪 RUNNING MODULE INTEGRATION TEST
Running unit tests...
🧪 Unit Test: Tensor Creation...
✅ Tensor creation works correctly!
🧪 Unit Test: Arithmetic Operations...
✅ Arithmetic operations work correctly!
🧪 Unit Test: Matrix Multiplication...
✅ Matrix multiplication works correctly!
🧪 Unit Test: Shape Manipulation...
✅ Shape manipulation works correctly!
🧪 Unit Test: Reduction Operations...
✅ Reduction operations work correctly!

Running integration scenarios...
🧪 Integration Test: Two-Layer Neural Network...
✅ Two-layer neural network computation works!
🧪 Integration Test: Complex Shape Operations...
✅ Complex shape operations work!
🧪 Integration Test: Broadcasting Edge Cases...
✅ Broadcasting edge cases work!

🎉 ALL TESTS PASSED! Module ready for export.
Run: tito module complete 01_tensor


# Demo tensor

In [13]:
def demo_tensor():
    """🎯 See your Tensor work just like NumPy."""
    print("🎯 AHA MOMENT: Your Tensor Works Like NumPy")
    print("=" * 45)

    # Create tensors
    a = Tensor(np.array([1, 2, 3]))
    b = Tensor(np.array([4, 5, 6]))

    # Tensor operations
    tensor_sum = a + b
    tensor_prod = a * b

    # NumPy equivalents
    np_sum = np.array([1, 2, 3]) + np.array([4, 5, 6])
    np_prod = np.array([1, 2, 3]) * np.array([4, 5, 6])

    print(f"Tensor a + b: {tensor_sum.data}")
    print(f"NumPy  a + b: {np_sum}")
    print(f"Match: {np.allclose(tensor_sum.data, np_sum)}")

    print(f"\nTensor a * b: {tensor_prod.data}")
    print(f"NumPy  a * b: {np_prod}")
    print(f"Match: {np.allclose(tensor_prod.data, np_prod)}")

    print("\n✨ Your Tensor is NumPy-compatible—ready for ML!")

if __name__ == "__main__":
    test_module()
    print("\n")
    demo_tensor()

🧪 RUNNING MODULE INTEGRATION TEST
Running unit tests...
🧪 Unit Test: Tensor Creation...
✅ Tensor creation works correctly!
🧪 Unit Test: Arithmetic Operations...
✅ Arithmetic operations work correctly!
🧪 Unit Test: Matrix Multiplication...
✅ Matrix multiplication works correctly!
🧪 Unit Test: Shape Manipulation...
✅ Shape manipulation works correctly!
🧪 Unit Test: Reduction Operations...
✅ Reduction operations work correctly!

Running integration scenarios...
🧪 Integration Test: Two-Layer Neural Network...
✅ Two-layer neural network computation works!
🧪 Integration Test: Complex Shape Operations...
✅ Complex shape operations work!
🧪 Integration Test: Broadcasting Edge Cases...
✅ Broadcasting edge cases work!

🎉 ALL TESTS PASSED! Module ready for export.
Run: tito module complete 01_tensor


🎯 AHA MOMENT: Your Tensor Works Like NumPy
Tensor a + b: [5. 7. 9.]
NumPy  a + b: [5 7 9]
Match: True

Tensor a * b: [ 4. 10. 18.]
NumPy  a * b: [ 4 10 18]
Match: True

✨ Your Tensor is NumPy-compati